<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

## Lab 4.2.2: Feature Selection

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

%matplotlib inline

### 5. Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

#### 5.1 Load Wine Data & Define Predictor and Target

In [32]:
## Load the wine quality dataset

# Load the wine dataset from csv
wine = pd.read_csv('E:/IOD_data/winequality_merged.csv')

# define the target variable (dependent variable) as y
y = wine['quality']

# Take all columns except target as predictor columns
predictor_columns = [c for c in wine.columns if c != 'quality']
# Load the dataset as a pandas data frame
X = pd.DataFrame(wine, columns = predictor_columns)

In [33]:
## Create training and testing subsets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#### 5.2 Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

#### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

#### Methods of the `LinearRegression` object to investigate
- `fit()`
- `fit.score()`

#### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

#### Linear Regression [reference](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [34]:
## Flag intermediate output

show_steps = True   # for testing/debugging
# show_steps = False  # without showing steps

In [35]:
X_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red_wine
4856,7.4,0.19,0.31,14.5,0.045,39.0,193.0,0.99860,3.10,0.50,9.200000,0
1779,6.4,0.33,0.31,5.5,0.048,42.0,173.0,0.99510,3.19,0.66,9.300000,0
684,9.8,0.98,0.32,2.3,0.078,35.0,152.0,0.99800,3.25,0.48,9.400000,1
4593,6.8,0.18,0.28,9.8,0.039,29.0,113.0,0.99406,3.11,0.45,10.900000,0
1128,10.0,0.43,0.33,2.7,0.095,28.0,89.0,0.99840,3.22,0.68,10.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
986,9.8,0.34,0.39,1.4,0.066,3.0,7.0,0.99470,3.19,0.55,11.400000,1
5971,7.1,0.23,0.23,3.5,0.038,23.0,112.0,0.99157,3.05,0.37,11.366667,0
904,6.8,0.59,0.06,6.0,0.060,11.0,18.0,0.99620,3.41,0.59,10.800000,1
6001,6.8,0.43,0.26,5.2,0.043,40.0,176.0,0.99116,3.17,0.41,12.300000,0


In [31]:
X_train.head()

,total sulfur dioxide
3587,201.0
1384,84.0
1719,87.0
5270,95.0
4677,91.0


In [37]:
## Use Forward Feature Selection to pick a good model

# start with no predictors
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
model = LinearRegression() # create a model for Linear Regression
# get the number of cases in the test data
n = X_test.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print('(Step) Trying %s...' % new_column)
            print('(Step) - Features = %s' % ', '.join(included + [new_column]))

        # fit the model with the Training data
        included.append(new_column)
        X_train_fs = X_train[included]
        X_test_fs = X_test[included]
        
        fit = model.fit(X_train_fs, y_train) # fit a model; consider which predictors should be included
        # calculate the score (R^2 for Regression)
        r2 = model.score(X_train_fs, y_train)# calculate the score
        # number of predictors in this model
        k = len(included) + 1
        # calculate the adjusted R^2
        adjusted_r2 = 1 - (((1-r2)*(n-1))/(n-k-1))# calculate the Adjusted R^2

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
              (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break

print('')
print('Resulting features:')
print(', '.join(included))


(Step) Excluded = total sulfur dioxide, red_wine, sulphates, density, pH, alcohol, residual sugar, volatile acidity, free sulfur dioxide, citric acid, chlorides, fixed acidity
(Step) Trying total sulfur dioxide...
(Step) - Features = total sulfur dioxide
(Step) - Adjusted R^2: This = 0.001; Best = 0.000
(Step) - New Best!   : Feature = total sulfur dioxide; R^2 = 0.002; Adjusted R^2 = 0.001
(Step) Trying red_wine...
(Step) - Features = total sulfur dioxide, red_wine
(Step) - Adjusted R^2: This = 0.049; Best = 0.001
(Step) - New Best!   : Feature = red_wine; R^2 = 0.051; Adjusted R^2 = 0.049
(Step) Trying sulphates...
(Step) - Features = total sulfur dioxide, red_wine, sulphates
(Step) - Adjusted R^2: This = 0.066; Best = 0.049
(Step) - New Best!   : Feature = sulphates; R^2 = 0.069; Adjusted R^2 = 0.066
(Step) Trying density...
(Step) - Features = total sulfur dioxide, red_wine, sulphates, density
(Step) - Adjusted R^2: This = 0.117; Best = 0.066
(Step) - New Best!   : Feature = densi



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



